In [4]:
import ezdxf
from ezdxf.addons.drawing import matplotlib
from ezdxf.addons.drawing import RenderContext, Frontend
from ezdxf.addons.drawing.matplotlib import MatplotlibBackend
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# レンズを定義
class Surface:
    def __init__(self, r, cd):
        self.radius = r
        self.clear_diameter = cd

class Singlet:
    def __init__(self, r1, cd1, r2, cd2, m, t, ed = None):
        self.left_surface  = Surface(r1, cd1)
        self.right_surface = Surface(r2, cd2)
        self.material = "N-BK7"
        self.thickness = t
        if ed is None:
            self.edge_diameter = max(cd1, cd2)
        else:
            self.edge_diameter = ed

        
lens = Singlet(38.88, 12, -40.00, 12, 'N-BK7', 2, 12)        


In [3]:
doc = ezdxf.new("R2000", setup=True, units = 4)
msp = doc.modelspace()

layers = ["structure", "dimension", "text", "frame"] #構造物 寸法 文字列 図枠
for layer in layers:
    doc.layers.add(name=layer)#特に属性は指定しない 

In [7]:
def add_line(p1, p2):
    new_line = msp.add_line(p1, p2, dxfattribs={'layer':'structure'})
    return new_line

def add_arc(vertex_p, R, D):
    center_p = (vertex_p[0]+radius, vertex_p[1])
    h = D/2
    start_ang = 0
    end_ang   = 0
    if R > 0:
        start_ang = 180 - np.degrees( np.arcsin(h/R) )
        end_ang   = 180 + start_ang
    else:
        start_ang = - np.degrees( np.arcsin(h/R) )
        end_ang   =  - start_ang
    new_arc = msp.add_arc(center=center_p, radius=R, start_angle=start_ang, end_angle=end_ang, dxfattribs={'layer':'structure'})
    return new_arc
 
    

In [ ]:
def draw_lens(singlet, pos):
    left_pos = pos
    add_arc(left_pos, singlet.left_surface.radius, singlet.left_surface.clear_diameter)
    right_pos = pos
    right_pos[0] += singlet.thickness
    add_arc(right_pos, singlet.right_surface.radius, singlet.right_surface.clear_diameter)
    
    